In [186]:
import os 
import json
import pprint
import pandas as pd
from pandas import json_normalize
import numpy as np

from IPython.display import display
#from pygnuplot import gnuplot


In [187]:

def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for x in d:
        for k, v in x.items():
            new_key = parent_key + sep + k if parent_key else k
            if isinstance(v, dict):
                items.extend(flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)

In [190]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



def parseQlogFile(qlog_file_path, regex):
    f = open(qlog_file_path, 'r')
    qlog_data = f.read()
    f.close()

    records = qlog_data.strip().split(regex)
    parsedRecords = parseQlogDataRecords(records)
    #data_dict = createDictionary(parsedRecords)
    transport_records = [record for record in parsedRecords[1:] if 'packet_sent' in record["name"] or 'packet_received' in record["name"] ]
   
    #displayPandasTable(parsedRecords)
    df = pd.DataFrame(transport_records).fillna('')
    #mux = pd.MultiIndex.from_tuples(r["data"].keys() for r in transport_records)
    #df = pd.DataFrame(list(r["data"].values() for r in transport_records), index=mux).fillna('')
    #display(df)
    handshake_time = getHandshakeTime(transport_records)
    
    return handshake_time
    
    #df = pd.MultiIndex(levels=[['zero', 'one'], ['x','y']], labels=[[1,1,0,],[1,0,1,]])

    #df = pd.DataFrame(data_dict)
    #df = pd.concat({k: pd.DataFrame(v).T for k, v in data_dict.items()}, axis=0)

    #df = pd.DataFrame.from_dict(flat_dict, orient="index")
    #df.index = pd.MultiIndex.from_tuples(df.index)
    #df = df.unstack(level=-1)
    #df.columns = df.columns.map("{0[1]}".format)

    #df = pd.DataFrame.from_dict(parsedRecords[0], orient='index')

def getHandshakeTime(records):
    packet_types = {}
    handshake_end_time = 0
    initial_packet_time = records[0]["time"]

    # Get time of first data packet:
    for record in records:
        packet_type = record["data"]["header"]["packet_type"]
        if packet_type != "initial" and packet_type != "handshake":
            handshake_end_time = record["time"]
            break

    # Get number of different packet types
    for record in records:
        packet_type = record["data"]["header"]["packet_type"]
        
        if packet_type not in packet_types:
            packet_types[packet_type] = 1
        else:
            packet_types[packet_type] += 1
    
    if len(packet_types.keys()) == 1:
        print("No connection was established.")
        return None
    else:
        print("Transport Packet Types: ", packet_types)
        print("Time to Complete Handshake: ", handshake_end_time - initial_packet_time, "ms")
        return handshake_end_time - initial_packet_time



def parseQlogDataRecords(qlog_data):
   json_data = [json.loads(string.strip()) for string in qlog_data]
   return json_data

def createDictionary(parsed_data):
    data_dict = {}

    for data in parsed_data:
        if type(data) == dict:
            for key, value in data.items():
                if key not in data_dict:
                    data_dict[key] = [value]
                else:
                    if type(value) == dict:
                        data_dict[key].append(createDictionary(value))
                    else:
                        data_dict[key].append(value)
        else:
            return parsed_data

    
    #pprint.pprint(data_dict)
    return data_dict

def displayPandasTable(parsedRecords):
    """
    for d in parsedRecords:
        flattened_data = createDictionary(d)
        
        df = pd.DataFrame(flattened_data)
        df = df.fillna('')
        print(df)
    """

    flattened_header_data = createDictionary(parsedRecords[0])
    flattened_header_data2 = createDictionary(parsedRecords[1])
    flattened_data = createDictionary(parsedRecords[1:])

    #pprint.pprint(flattened_data)
            
    # Create DataFrame from flattened data
    #df = pd.DataFrame(parsedRecords)
    #df = pd.concat({k: pd.DataFrame(v).T for k, v in data_dict.items()}, axis=0)
    df1 = pd.DataFrame(flattened_header_data)
    df2 = pd.DataFrame(flattened_header_data2)
    df2 = pd.DataFrame(flattened_header_data2)
    #df = pd.DataFrame({k: [flattened_data.get(k, None)] for k in keys})
    df = pd.DataFrame(flattened_data)

    df = df.fillna('')
    # Display DataFrame
    #df = json_normalize(df)
    #df = pd.MultiIndex.from_frame(df)
    #df.to_flat_index()
    """
    display(df1)
    display(df2)
    display(df)
    """

def getMeanTime(times):
    return np.mean([time for time in times if time != None])



In [193]:

regex = chr(30)
# Folder Path 
path = "C:/Users/larad/Documents/l4project/data/packetCapture_home/quic"
# Change the directory 
os.chdir(path) 
ctr = 0

handshake_times = []
#qlog_files=0
# iterate through all file 
for folder in os.listdir(): 
    #print("inside " + folder)
    quic_folder_path = f"{path}/{folder}"
    os.chdir(quic_folder_path)
    
    #if folder == 'www.youtube.com':
    for file in os.listdir():
        #print("inside for")
        # Check whether file is in sqlog format or not 
        if file.endswith(".sqlog"): 
            handshake_times.append(parseQlogFile(f"{quic_folder_path}/{file}", regex))
            #print("file "+qlog_file_path+" has been opened")
            #ctr += 1
            #qlog_files += 1

    
print("Average time to complete handshake: ", getMeanTime(handshake_times), "ms")

No connection was established.
Transport Packet Types:  {'initial': 4, 'handshake': 6, '1RTT': 145}
Time to Complete Handshake:  229.576549002 ms
No connection was established.
No connection was established.
No connection was established.
Transport Packet Types:  {'initial': 4, 'handshake': 7, '1RTT': 18}
Time to Complete Handshake:  162.384712 ms
No connection was established.
No connection was established.
No connection was established.
Transport Packet Types:  {'initial': 5, 'handshake': 4, '1RTT': 131}
Time to Complete Handshake:  54.79692 ms
No connection was established.
No connection was established.
No connection was established.
No connection was established.
No connection was established.
Transport Packet Types:  {'initial': 4, 'handshake': 10, '1RTT': 96}
Time to Complete Handshake:  28.716226000000002 ms
No connection was established.
Transport Packet Types:  {'initial': 5, 'handshake': 6, '1RTT': 52}
Time to Complete Handshake:  30.972584004 ms
No connection was establishe